### Requirements

In [ ]:
!pip install arm-mango

In [ ]:
!pip install torcheck #### Check for potential issue in models

In [ ]:
!pip install ogb

### Load data

In [ ]:
import pandas as pd

DATA_PATH = "pre-processed/fixtures_full.csv"
data = pd.read_csv(DATA_PATH)
data['result'].value_counts()

In [ ]:
import numpy as np
np.where(data.applymap(lambda x: x == ''))

In [ ]:
np.where(pd.isnull(data))

### GNN
Using hierachical pooling to reduce the graph until it's only one embedding
- Hierachical pooling: reduce the nodes by distribute node to neighbor nodes

In [1]:
!mlflow ui

^C


### GNN

In [ ]:
%%writefile model.py
import torch
import torch.nn.functional as F 
from torch.nn import Linear, BatchNorm1d, ModuleList
from torch_geometric.nn import TransformerConv, TopKPooling 
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.nn import BatchNorm, PNAConv, global_add_pool
#torch.manual_seed(42)

class GNN(torch.nn.Module):
    def __init__(self, feature_size, model_params):
        super(GNN, self).__init__()
        embedding_size = model_params["model_embedding_size"]
        n_heads = model_params["model_attention_heads"]
        self.n_layers = model_params["model_layers"]
        dropout_rate = model_params["model_dropout_rate"]
        top_k_ratio = model_params["model_top_k_ratio"]
        self.top_k_every_n = model_params["model_top_k_every_n"]
        dense_neurons = model_params["model_dense_neurons"]
        #edge_dim = model_params["model_edge_dim"]
        

        self.conv_layers = ModuleList([])
        self.transf_layers = ModuleList([])
        self.pooling_layers = ModuleList([])
        self.bn_layers = ModuleList([])

        # Transformation layer: transform original node features to embedding vector(size: embedding_size(defined in config.py))
        self.conv1 = TransformerConv(feature_size, 
                                    embedding_size, 
                                    heads=n_heads, 
                                    dropout=dropout_rate,
                                    #edge_dim=edge_dim,
                                    beta=True) 

        self.transf1 = Linear(embedding_size*n_heads, embedding_size)
        self.bn1 = BatchNorm1d(embedding_size)

        # Other layers: message passing and pooling
        for i in range(self.n_layers):
            self.conv_layers.append(TransformerConv(embedding_size, 
                                                    embedding_size, 
                                                    heads=n_heads, 
                                                    dropout=dropout_rate,
                                                    #edge_dim=edge_dim,
                                                    beta=True))

            # map conv_layer output size back to emgedding_size(embedding_size*n_heads -> embedding_size)
            self.transf_layers.append(Linear(embedding_size*n_heads, embedding_size))
            # Batch normalization
            self.bn_layers.append(BatchNorm1d(embedding_size))
            # Top-k pooling to reduce the size of the graph
            if i % self.top_k_every_n == 0:
                self.pooling_layers.append(TopKPooling(embedding_size, ratio=top_k_ratio))
            

        # Linear output layers: feed graph representation in & reduce until single value left
        self.linear1 = Linear(embedding_size*2, dense_neurons)
        self.linear2 = Linear(dense_neurons, int(dense_neurons/2))  
        #self.linear3 = Linear(int(dense_neurons/2), 1)  
        self.linear3 = Linear(int(dense_neurons/2), 3) # same as the general form

    #def forward(self, x, edge_index):
    def forward(self, x, edge_index, batch_index):
    #def forward(self, x, edge_attr=None, edge_index, batch_index):
        # Initial transformation
        #x = self.conv1(x, edge_index, edge_attr)
        x = self.conv1(x, edge_index)
        x = torch.relu(self.transf1(x))
        x = torch.relu((x))

        x = self.bn1(x)

        # Holds the intermediate graph representations
        global_representation = []

        for i in range(self.n_layers):
            #x = self.conv_layers[i](x, edge_index, edge_attr)
            x = self.conv_layers[i](x, edge_index)
            x = torch.relu(self.transf_layers[i](x))
            x = torch.relu((x))
            x = self.bn_layers[i](x)
            # Always aggregate last layer
            if i % self.top_k_every_n == 0 or i == self.n_layers:
                x , edge_index, edge_attr, batch_index, _, _ = self.pooling_layers[int(i/self.top_k_every_n)](  x, 
                                                                                                                edge_index, 
                                                                                                                None, 
                                                                                                                batch_index)              
                # Add current representation
                global_representation.append(torch.cat([gmp(x, batch_index), gap(x, batch_index)], dim=1))
    
        x = sum(global_representation)

        # Output block
        x = torch.relu(self.linear1(x))
        x = F.dropout(x, p=0.8, training=self.training)
        x = torch.relu(self.linear2(x))
        x = F.dropout(x, p=0.8, training=self.training)
        x = self.linear3(x)

        return x

#### Train Epoch, Test, Log

In [ ]:
#imports 
import torch 
from torch_geometric.data import DataLoader
from sklearn.metrics import confusion_matrix, f1_score, \
    accuracy_score, precision_score, recall_score, roc_auc_score
from processedDataset import ProcessedDataset
import numpy as np
from tqdm import tqdm
from model import GNN
import mlflow.pytorch
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Specify tracking server
#mlflow.set_tracking_uri("http://localhost:5000")

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train_one_epoch(epoch, model, train_loader, optimizer, loss_fn):
    # Enumerate over the data
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for _, batch in enumerate(tqdm(train_loader)):
        batch.x = torch.tensor(batch.x)
        batch.x = batch.x.reshape((-1, *batch.x.shape[2:]))
        #print("this is batch.x shape: ", batch.x.shape)
        # Use GPU
        batch.to(device)  
        # Reset gradients
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        pred = model(torch.tensor(batch.x).float(), 
                                #batch.edge_attr.float(),
                                batch.edge_index, 
                                batch.batch) 
        # Calculating the loss and gradients
        #loss = loss_fn(torch.squeeze(pred), batch.y.float()) # has error if loss_fn changed to cross_entropy
        loss = torch.sqrt(loss_fn(pred, batch.y.long()))

        loss.backward() 
        optimizer.step()
        
        # Update tracking
        running_loss += loss.item()
        step += 1
        
#         predict = torch.sigmoid(pred).cpu().detach().numpy()

        #all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        
        all_labels.append(batch.y.cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    calculate_metrics(all_preds, all_labels, epoch, "train")
    return running_loss/step

def test(epoch, model, test_loader, loss_fn):
    all_preds = []
    all_preds_raw = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for batch in test_loader:
        batch.x = torch.tensor(batch.x)
        batch.x = batch.x.reshape((-1, *batch.x.shape[2:]))
        batch.to(device)  
        pred = model(torch.tensor(batch.x).float(), 
                        #batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch) 
        #loss = loss_fn(torch.squeeze(pred), batch.y.float()) # has error if loss_fn changed to cross_entropy
        loss = torch.sqrt(loss_fn(pred, batch.y.long()))
         # Update tracking
        running_loss += loss.item()
        step += 1
        
#         predict = torch.sigmoid(pred).cpu().detach().numpy()
        
        #all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    print(all_preds_raw[0][:10])
    print(all_preds[:10])
    print(all_labels[:10])
    calculate_metrics(all_preds, all_labels, epoch, "test")
    log_conf_matrix(all_preds, all_labels, epoch)
    return running_loss/step

def final_test(epoch, model, final_test_loader, threshold):
    all_preds = []
    all_preds_raw = []
    all_labels = []

    for batch in final_test_loader:
        batch.x = torch.tensor(batch.x)
        batch.x = batch.x.reshape((-1, *batch.x.shape[2:]))
        batch.to(device)  
        pred = model(torch.tensor(batch.x).float(), 
                        #batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch) 

        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    
    # Pick the top N confidence
    num_threshold = int(len(all_preds) * threshold)
    ind = all_preds # (463, )
    raw_nums = np.array(sum([l.tolist() for l in all_preds_raw], [])) #( 463 * 3)
    conf_nums = [] # the confidence of that predicted label ex. ind = 2 & raw_nums = [0.4, 0.45, 0.5] -> 0.5
    for i, list_ in zip(ind, raw_nums):
        conf_nums.append(list_[i])
    #print("conf_nums: ", conf_nums)
    
    # get the top N conf_nums' indices
    indices = np.argpartition(conf_nums, -num_threshold)[-num_threshold:]
    #print("indices: ", indices)
    top_preds = all_preds[indices]
    top_labels = all_labels[indices]
    #print("top_preds: ", top_preds, "| top_labels: ", top_labels)
    
    # calculate acc
    acc = accuracy_score(top_labels, top_preds)
    mlflow.log_metric(key="Final Test Accuracy", value=float(acc), step=epoch)

    print("final acc: ", acc)

def log_conf_matrix(y_pred, y_true, epoch):
    # Log confusion matrix as image
    cm = confusion_matrix(y_true, y_pred)
    classes = ["0", "1", "2"]
    df_cfm = pd.DataFrame(cm, index = classes, columns = classes)
    plt.figure(figsize = (10,7))
    cfm_plot = sns.heatmap(df_cfm, annot=True, cmap='Blues', fmt='g')
    cfm_plot.set_ylabel('True label')
    cfm_plot.set_xlabel('Predicted label')
    cfm_plot.figure.savefig(f'data/images/cm_{epoch}.png')
    mlflow.log_artifact(f"data/images/cm_{epoch}.png")

def calculate_metrics(y_pred, y_true, epoch, type):
    try:
        print(f"\n Confusion matrix: \n {confusion_matrix(y_pred, y_true)}")##############################
    except:
        print("Error with confusion matrix")
    f1 = f1_score(y_true, y_pred, average='weighted')
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted')
    rec = recall_score(y_true, y_pred, average='weighted')
    print(f"F1 Score: {f1}")
    print(f"Accuracy: {acc}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    mlflow.log_metric(key=f"F1-{type}", value=float(f1), step=epoch)
    mlflow.log_metric(key=f"Accuracy-{type}", value=float(acc), step=epoch)
    mlflow.log_metric(key=f"Precision-{type}", value=float(prec), step=epoch)
    mlflow.log_metric(key=f"Recall-{type}", value=float(rec), step=epoch)
    # try:
    #     roc = roc_auc_score(y_true, y_pred)
    #     print(f"ROC AUC: {roc}")
    #     mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(roc), step=epoch)
    # except:
    #     #mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(0), step=epoch)
    #     print(f"ROC AUC: notdefined")
    
def predict(model, test_loader):
    all_preds = []
    all_preds_raw = []
    all_labels = []

    for batch in test_loader:
        batch.x = torch.tensor(batch.x)
        batch.x = batch.x.reshape((-1, *batch.x.shape[2:]))
        batch.to(device)  
        pred = model(torch.tensor(batch.x).float(), 
                        #batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch) 

        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    return all_preds, all_preds_raw, all_labels

In [ ]:
# Run the training
from mango import scheduler, Tuner
from config import HYPERPARAMETERS, BEST_PARAMETERS, SIGNATURE
from sklearn.utils import class_weight
from torch.utils.data import random_split
from torch.utils.data.sampler import SubsetRandomSampler
from torch_geometric.utils import degree

torch.manual_seed(42)

def run_one_training(params):
    params = params[0]
    with mlflow.start_run() as run:
        # Log parameters used in this experiment
        for key in params.keys():
            mlflow.log_param(key, params[key])

        # Loading the dataset
        # Graph-level tasks: Graph classification - https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial7/GNN_overview.html#Graph-level-tasks:-Graph-classification 
        print("Loading dataset...")
        full_dataset = ProcessedDataset(root = "data/", filename = "fixtures_full.csv")
        full_dataset.shuffle()
        train_dataset = full_dataset[:3400] # around 80% of the full dataset
        test_dataset = full_dataset[3400:3800]
        final_test_dataset = full_dataset[3800:]
        
        #train_size = int(0.8 * len(full_dataset))
        #test_size = len(full_dataset) - train_size
        #print("size of train and test: ", train_size, test_size)
        #train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
        #print(test_dataset)
        
        # Prepare training
        print("Preparing Training")
        batch_size=params["batch_size"]
        train_loader = DataLoader(train_dataset, batch_size=batch_size)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)  
        final_test_loader = DataLoader(final_test_dataset, batch_size=batch_size)
        
        # # for checking dataloader
        # dataiter = iter(train_loader)
        # data = next(dataiter)
        # features, labels, aa, *_ = data
        # print("features: ", features)
        # print("labels: ", labels)
        # print("aa: ", aa)
        # print(np.array(features[1]).shape)
        # print(labels[0])
        # print(labels[1].shape)
        # print(aa[1])
        
        # Loading the model
        print("Loading model...")
        model_params = {k: v for k, v in params.items() if k.startswith("model_")}
        model = GNN(feature_size=train_dataset[0].x.shape[1], model_params=model_params)

        print(model)
        model = model.to(device)
        print(f"Number of parameters: {count_parameters(model)}")
        mlflow.log_param("num_params", count_parameters(model))

        # < 1 increases precision, > 1 recall
        #weight = torch.tensor([params["pos_weight"]], dtype=torch.float32).to(device)
        #loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=weight)
        
#         # add in weight of class according to train dataset
#         train_indices = train_dataset.indices
        
#         DATA_PATH = "pre-processed/fixtures_full.csv"
#         total = pd.read_csv(DATA_PATH)
#         train_ds_bincount = np.bincount(total.iloc[train_indices]['result'])
#         #print(train_ds_bincount)

#         target = total.iloc[train_indices]['result']
#         #print(target)
#         class_weights= class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(target), y=target)
#         class_weights= torch.tensor(class_weights,dtype=torch.float)

        #print(class_weights) # etc. ([1.0239, 1.2753, 0.8070])
        class_weights = [1.0239, 1.2753, 0.8070]
        class_weights= torch.tensor(class_weights,dtype=torch.float)
        
        #loss_fn = torch.nn.CrossEntropyLoss()
        loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
        optimizer = torch.optim.SGD(model.parameters(), 
                                    lr=params["learning_rate"], 
                                    weight_decay=params["weight_decay"], 
                                    momentum=params["sgd_momentum"])
        # optimizer = torch.optim.AdamW(model.parameters(), 
        #                               lr=params["learning_rate"], 
        #                               weight_decay=params["weight_decay"])
        
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=params["scheduler_gamma"])
        
        # Start training
        best_loss = 1000
        early_stopping_counter = 0
        for epoch in range(3000): 
            if early_stopping_counter <= 20: # = x * 5 
                # Training
                model.train()
                loss = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
                print(f"Epoch {epoch} | Train Loss {loss}")
                mlflow.log_metric(key="Train loss", value=float(loss), step=epoch)

                # Testing
                model.eval()
                if epoch % 5 == 0:
                    loss = test(epoch, model, test_loader, loss_fn)
                    print(f"Epoch {epoch} | Test Loss {loss}")
                    mlflow.log_metric(key="Test loss", value=float(loss), step=epoch)
                    
                    # Update best loss
                    if float(loss) < best_loss:
                        best_loss = loss
                        # Save the currently best model 
                        mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)
                        early_stopping_counter = 0
                    else:
                        early_stopping_counter += 1

                scheduler.step()
            else:
                print("Early stopping due to no improvement.")
                # model.eval()
                # final_test(epoch, model, final_test_loader, threshold=params["threshold"])
                return [best_loss]
        print(f"Finishing training with best test loss: {best_loss}")
        # model.eval()
        # final_test(3000, model, final_test_loader, threshold=params["threshold"])
        return [best_loss]

In [ ]:
# Hyperparameter search
print("Running hyperparameter search...")
config = dict()
config["optimizer"] = "Bayesian"
config["num_iteration"] = 100

tuner = Tuner(HYPERPARAMETERS, 
              objective=run_one_training,
              conf_dict=config) 
results = tuner.minimize()

In [ ]:
import mlflow
import torch
import torch_geometric

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

logged_model = 'runs:/de621c161db0430db8ed68cb52120e8f/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pytorch.load_model(logged_model)
for conv in loaded_model.conv_layers:
    conv.aggr_module = torch_geometric.nn.aggr.SumAggregation()

loaded_model = loaded_model.to(device)
loaded_model.eval()

In [ ]:
from processedDataset import ProcessedDataset
from torch_geometric.data import DataLoader

full_dataset = ProcessedDataset(root = "data/", filename = "fixtures_full.csv")
full_dataset.shuffle()
dataset = full_dataset[3800:]

loader = DataLoader(dataset, batch_size=len(dataset))

all_pred, all_pred_raw, all_label = predict(loaded_model, loader)

### General Version

In [ ]:
# check mlflow
!mlflow --version

In [ ]:
%%writefile model.py
import torch
import torch.nn.functional as F 
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU
from torch_geometric.nn import TransformerConv, GATConv, TopKPooling, BatchNorm
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
torch.manual_seed(42)

class GNN(torch.nn.Module):
    def __init__(self, feature_size):
        super(GNN, self).__init__()
        num_classes = 3
        embedding_size = 1024

        # GNN layers
        self.conv1 = GATConv(feature_size, embedding_size, heads=3, dropout=0.3)
        self.head_transform1 = Linear(embedding_size*3, embedding_size)
        self.pool1 = TopKPooling(embedding_size, ratio=0.8)
        self.conv2 = GATConv(embedding_size, embedding_size, heads=3, dropout=0.3)
        self.head_transform2 = Linear(embedding_size*3, embedding_size)
        self.pool2 = TopKPooling(embedding_size, ratio=0.5)
        self.conv3 = GATConv(embedding_size, embedding_size, heads=3, dropout=0.3)
        self.head_transform3 = Linear(embedding_size*3, embedding_size)
        self.pool3 = TopKPooling(embedding_size, ratio=0.2)

        # Linear layers
        self.linear1 = Linear(embedding_size*2, 1024) # 1024 dense neurons
        self.linear2 = Linear(1024, num_classes)   

    def forward(self, x, edge_index, batch_index):
        # def forward(self, x, edge_attr, edge_index, batch_index):
        # First block
        x = self.conv1(x, edge_index)
        x = self.head_transform1(x)

        x, edge_index, edge_attr, batch_index, _, _ = self.pool1(x, 
                                                                 edge_index,
                                                                 None,
                                                                 batch_index)
        x1 = torch.cat([gmp(x, batch_index), gap(x, batch_index)], dim=1)

        # Second block
        x = self.conv2(x, edge_index)
        x = self.head_transform2(x)
        x, edge_index, edge_attr, batch_index, _, _ = self.pool2(x,
                                                                edge_index,
                                                                None,
                                                                batch_index)
        x2 = torch.cat([gmp(x, batch_index), gap(x, batch_index)], dim=1)

        # Third block
        x = self.conv3(x, edge_index)
        x = self.head_transform3(x)
        x, edge_index, edge_attr, batch_index, _, _ = self.pool3(x,
                                                                edge_index,
                                                                None,
                                                                batch_index)
        x3 = torch.cat([gmp(x, batch_index), gap(x, batch_index)], dim=1)

        # Concat pooled vectors
        x = x1 + x2 + x3

        # Output block
        x = self.linear1(x).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.linear2(x)

        return x

##### Training

In [ ]:
# imports 
import torch 
from torch_geometric.data import DataLoader
from sklearn.metrics import confusion_matrix, f1_score, \
    accuracy_score, precision_score, recall_score, roc_auc_score
import numpy as np
from tqdm import tqdm
from processedDataset import ProcessedDataset
from model import GNN
import mlflow.pytorch
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
train_dataset = ProcessedDataset(root = "data/", filename = "fixtures_train.csv")
test_dataset = ProcessedDataset(root = "data/", filename = "fixtures_test.csv", test=True)

In [ ]:
model = GNN(feature_size=train_dataset[0].x.shape[1])
model = model.to(device)

print(f"Number of parameters: {count_parameters(model)}")
model

In [ ]:
# loss and optimizer, could try some more to see which is better
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9) # or ADAM...etc
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95) #

In [ ]:
# prepare training
NUM_GRAPHS_PER_BATCH = 256
train_loader = DataLoader(train_dataset, 
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_loader = DataLoader(test_dataset, 
                        batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)

def train(epoch):
    # Enumerate over the data
    all_preds = []
    all_labels = []
    for _, batch in enumerate(tqdm(train_loader)):
        # use CPU(or GPU)
        batch.to(device)
        # Reset gradients
        optimizer.zero_grad()
        # passing the node features and the connection info
        pred = model(batch.x.float(), 
                   # batch.edge_attr.float(), 
                    batch.edge_index, 
                    batch.batch)
        # calculating the loss and gradients
        loss = torch.sqrt(loss_fn(pred, batch.y))
        loss.backward()
        # update using the gradients
        optimizer.step()
        
        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_labels.append(batch.y.cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    calculate_metrics(all_preds, all_labels, epoch, "train")
    return loss

def test(epoch):
    all_preds = []
    all_labels = []
    for batch in test_loader:
        batch.to(device)
        pred = model(batch.x.float(), 
                   # batch.edge_attr.float(), 
                    batch.edge_index, 
                    batch.batch)
        loss = torch.sqrt(loss_fn(pred, batch.y))
        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_labels.append(batch.y.cpu().detach().numpy())
        
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    calculate_metrics(all_preds, all_labels, epoch, "test")
    return loss

def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"\n Confusion matrix: \n {confusion_matrix(y_pred, y_true)}")
    print(f"F1 Score: {f1_score(y_pred, y_true, average='macro')}")
    print(f"Accuracy: {accuracy_score(y_pred, y_true)}")
    print(f"Precision: {precision_score(y_pred, y_true, average='macro')}")
    print(f"Recall: {recall_score(y_pred, y_true, average='macro')}")
    try:
        roc = roc_auc_score(y_pred, y_true)
        print(f"ROC AUC: {roc}")
        mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(roc), step=epoch)
    except:
        mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(0), step=epoch)
        print(f"ROC AUC: notdefined")

In [ ]:
# run the training
# Run the training
with mlflow.start_run() as run:
    for epoch in range(500): 
        # Training
        model.train()
        loss = train(epoch=epoch)
        loss = loss.detach().cpu().numpy()
        print(f"Epoch {epoch} | Train Loss {loss}")
        mlflow.log_metric(key="Train loss", value=float(loss), step=epoch)

        # Testing
        model.eval()
        if epoch % 5 == 0:
            loss = test(epoch=epoch)
            loss = loss.detach().cpu().numpy()
            print(f"Epoch {epoch} | Test Loss {loss}")
            mlflow.log_metric(key="Test loss", value=float(loss), step=epoch)

        scheduler.step()

    print("Done.")


In [ ]:
# Save the model
mlflow.pytorch.log_model(model, "model")

In [ ]:
!mlflow ui